# Chapter 2: Pedometer and calorie tracker  

Pedometers are popular as an everyday exercise progress monitor and motivator. We look at the steps to go from 3-axis accelerometer readings to counting steps. Note that there are multiple ways to approach this problem, and what is described below is one approach.

In [1]:
import math
import numpy as np

## Converting a 3-axis accelerometer signal into a 1-dimensional signal

The first step is somehow converting a 3-axis accelerometer signal into a one-dimensional signal that we can use for detecting peaks. 

In [2]:
def pull_data(dir_name, file_name):
    f = open(dir_name + '/' + file_name + '.csv')
    xs = []
    ys = []
    zs = []
    rs = []
    timestamps = []
    for line in f:
        value = line.split(',')
        if len(value) > 3:
            timestamps.append(float(value[-4]))
            x = float(value[-3])
            y = float(value[-2])
            z = float(value[-1])
            r = math.sqrt(x ** 2 + y ** 2 + z ** 2)
            xs.append(x)
            ys.append(y)
            zs.append(z)
            rs.append(r)
    return np.array(xs), np.array(ys), np.array(zs), np.array(rs), np.array(timestamps)


## Case study 1: Step Counting

Let us look at how to extract steps from the raw time-series. Let us first plot the three-axis accelerometer readings. 

In [3]:
# read accelerometer data file

accel_file = 'data/walk/inPocket/john1'
x_data, y_data, z_data, signal, timestamps = pull_data(accel_file, 'accelerometer')

c = timestamps[0]
timestamps = (timestamps - c)/1000000000

**ASSIGNMENT:** Plot acceleration readings along each of the three axes

In [4]:
# FILL IN CODE: plot three-axis acceleration

### Using magnitude rather than 3-axis signal

One issue with using the 3 axis signal directly is that the shape of the x, y, z curves depends on the orientation of the phone (or smartwatch). The acceleration changes as a result of the step can result in changes along all the three axes, so we need to design an orientation-independent algorithm. The easiest way to do this is to use the magnitude of the signal across all the three axes.

**ASSIGNMENT:** Plot the magnitude of acceleration

In [5]:
# FILL IN CODE: plot magnitude

### Cleaning the accelerometer magnitude signal with a butterworth filter

We are now going to use a frequency domain filter called a 'butterworth filter' to remove noise (see previous notebook for further explanation of frequency-domain filtering). A Butterworth filter is a popular frequency domain 'lowpass' filter that can remove high frequency noise while only letting the low frequencies through. Since the walking signal is low frequency (only a few steps minute), this is a good approach.

**ASSIGNMENT:** Apply a Butterworth filter (as demonstrated in the previous notebook) and plot both unfiltered and filtered accelerometer signals (magnitude only).

In [6]:
# FILL IN CODE: Create and apply Butterworth filter

In [7]:
# FILL IN CODE: Plot signals

### Counting steps

Our next problem is to determine when a step occurs. From looking at the figure, you could probably come up with a few potential algorithms. For example, we could extract the peak point of each step or look for crossings of the mean signal value. Note that the person seems to have stopped for a few seconds during his walk, where the signal is oscillating slightly about the mean value. It is important to carefully consider such changes since the threshold values could change depending on the current activity.

**ASSIGNMENT:** From the filtered signal, determine the number of steps taken by the person. You can use one of the abovementioned methods or implement your own algorithm to do so. Plot the filtered signal and the detected steps.

In [8]:
# FILL IN CODE: Step counting